In [1]:
%cd ~/ml-class/examples/keras-sign

/home/mb706/ml-class/examples/keras-sign


In [2]:
%pwd

'/home/mb706/ml-class/examples/keras-sign'

In [11]:
# A very simple perceptron for classifying american sign language letters
import signdata
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Reshape
from keras.utils import np_utils
import wandb
from wandb.keras import WandbCallback

# logging code
run = wandb.init()
config = run.config
config.loss = "categorical_crossentropy"
config.optimizer = "adam"
config.epochs = 10

# load data
(X_test, y_test) = signdata.load_test_data()
(X_train, y_train) = signdata.load_train_data()

E1029 17:06:08.184342 140089505888064 jupyter.py:104] Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.8.13 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [53]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [9]:
X_test.shape

(7172, 28, 28)

In [11]:
y_train  # classes; needs to be one-hot encoded

array([ 3,  6,  2, ..., 18, 17, 23])

In [12]:
img_width = X_test.shape[1]
img_height = X_test.shape[2]

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_train.shape[1]

In [5]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [6]:
Sequential.__module__

'keras.engine.sequential'

In [7]:
print(Flatten.__module__)
print(Dense.__module__)

keras.layers.core
keras.layers.core


In [47]:
??EarlyStopping

Init signature:
EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
)
Source:        
class EarlyStopping(Callback):
    """Stop training when a monitored quantity has stopped improving.

    # Arguments
        monitor: quantity to be monitored.
        min_delta: minimum change in the monitored quantity
            to qualify as an improvement, i.e. an absolute
            change of less than min_delta, will count as no
            improvement.
        patience: number of epochs with no improvement
            after which training will be stopped.
        verbose: verbosity mode.
        mode: one of {auto, min, max}. In `min` mode,
            training will stop when the quantity
            monitored has stopped decreasing; in `max`
            mode it will stop when the quantity
            monitored has stopped increasing; in `auto`
            mode, the direction is automatical

In [46]:
from keras.callbacks import EarlyStopping

In [51]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', min_delta = 0.01, patience = 3)

In [37]:
X_train = X_train.reshape((-1, 28, 28, 1))
X_test = X_test.reshape((-1, 28, 28, 1))

In [59]:
# create model
model = Sequential()  # 7172 x 28 x 28
model.add(Conv2D(32, (5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(num_classes * 4, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation="softmax"))
model.compile(loss=config.loss, optimizer=config.optimizer,
              metrics=['accuracy'])

# Fit the model
model.fit(X_train / 255, y_train, epochs=20, validation_data=(X_test / 255, y_test),
          callbacks=[WandbCallback(data_type="image", labels=signdata.letters), es])

Train on 27455 samples, validate on 7172 samples
Epoch 1/20
   32/27455 [..............................] - ETA: 16:47 - loss: 3.2353 - acc: 0.0625

E1029 18:05:20.497938 140089505888064 jupyter.py:104] Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.8.13 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  480/27455 [..............................] - ETA: 2:18 - loss: 3.2183 - acc: 0.0563

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.546142). Check your callbacks.
  % delta_t_median)


27455/27455 [==============================] - 16s 595us/step - loss: 1.9824 - acc: 0.3755 - val_loss: 0.7807 - val_acc: 0.7688
Epoch 2/20
27455/27455 [==============================] - 15s 533us/step - loss: 0.7181 - acc: 0.7548 - val_loss: 0.4231 - val_acc: 0.8606
Epoch 3/20
27455/27455 [==============================] - 14s 505us/step - loss: 0.4246 - acc: 0.8555 - val_loss: 0.2687 - val_acc: 0.9169
Epoch 4/20
27455/27455 [==============================] - 14s 502us/step - loss: 0.2962 - acc: 0.9009 - val_loss: 0.2639 - val_acc: 0.9165
Epoch 5/20
27455/27455 [==============================] - 14s 505us/step - loss: 0.2336 - acc: 0.9222 - val_loss: 0.1954 - val_acc: 0.9345
Epoch 6/20
27455/27455 [==============================] - 14s 502us/step - loss: 0.1797 - acc: 0.9394 - val_loss: 0.1386 - val_acc: 0.9525
Epoch 7/20
27455/27455 [==============================] - 14s 503us/step - loss: 0.1554 - acc: 0.9475 - val_loss: 0.1365 - val_acc: 0.9624
Epoch 8/20
27455/27455 [==============